In [36]:
import pandas as pd
import numpy as np
from numba import njit
from tqdm import tqdm

In [2]:
metadata = pd.read_csv('/mnt/10tb/home/shmelev/new_graph_and_metadata/data_anon.csv')

/mnt/10tb/home/shmelev/miniconda3/envs/genlink/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3308: DtypeWarning: Columns (4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
metadata

,node_id,ancestor_id,ancestor_level,through,node_birthplace_region,ancestor_gender,ancestor_birthplace_region,node_ibd_graph_id,ancestor_ibd_graph_id
0,2668455,2988210,0,0.0,NaN,Female,NaN,135785,-1
1,2668455,3533540,1,1.0,NaN,Male,NaN,135785,-1
2,2668455,4428401,1,0.0,NaN,Female,NaN,135785,-1
3,2668455,1652696,0,1.0,NaN,Male,NaN,135785,-1
4,2668455,2648119,1,11.0,NaN,Male,NaN,135785,-1
...,...,...,...,...,...,...,...,...,...
22907453,5234476,1968842,1,11.0,Санкт-Петербург,Male,NaN,-1,-1
22907454,5234476,1174620,1,10.0,Санкт-Петербург,Female,NaN,-1,-1
22907455,4008901,2395565,0,0.0,NaN,Female,Ярославская,-1,-1
22907456,4008901,4966205,1,1.0,NaN,Male,NaN,-1,-1


In [4]:
d1={
  "NaN": "Unknown",
  "Полтавская": "Ukranians",
  "Запорожская": "Ukranians",
  "Москва": "Northen Russians",
  "Санкт-Петербург": "Northen Russians",
  "Башкортостан": "Northen Russians",
  "Пермский": "Northen Russians",
  "Kamchatsky": "Northen Russians",
  "Karakalpakstan": "Unknown",
  "Тамбовская": "Northen Russians",
  "Херсонская": "Ukranians",
  "Крым": "Ukranians",
  "Тверская": "Northen Russians",
  "Восточно-Казахстанская": "Unknown",
  "Красноярский": "Northen Russians",
  "Донецкая": "Ukranians",
  "Львовская": "Ukranians",
  "Татарстан": "Northen Russians",
  "Новосибирская": "Northen Russians",
  "Черкасская": "Ukranians",
  "Челябинская": "Northen Russians",
  "Московская": "Northen Russians",
  "Рязанская": "Northen Russians",
  "Ростовская": "Southern Russians",
  "Свердловская": "Northen Russians",
  "Курская": "Northen Russians",
  "Самарская": "Northen Russians",
  "Shamkir Rayon": "Unknown",
  "Краснодарский": "Southern Russians",
  "Иркутская": "Northen Russians",
  "Коми": "Northen Russians",
  "Псковская": "Northen Russians",
  "Саратовская": "Northen Russians",
  "Омская": "Northen Russians",
  "Тульская": "Northen Russians",
  "Забайкальский": "Northen Russians",
  "Архангельская": "Northen Russians",
  "Astara": "Unknown",
  "Днепропетровская": "Ukranians",
  "Сумская": "Ukranians",
  "Ульяновская": "Northen Russians",
  "Павлодарская": "Unknown",
  "Дагестан": "Southern Russians",
  "Липецкая": "Northen Russians",
  "Пензенская": "Northen Russians",
  "Вологодская": "Northen Russians",
  "Ярославская": "Northen Russians",
  "Мордовия": "Northen Russians"
}


In [5]:
d2={
  "Калининградская": "Northen Russians",
  "Хабаровский": "Northen Russians",
  "Кировская": "Northen Russians",
  "Ямало-Ненецкий": "Northen Russians",
  "Qazax": "Unknown",
  "Agstafa": "Unknown",
  "Оренбургская": "Northen Russians",
  "Baki": "Unknown",
  "Минская": "Belarusians",
  "Moscow": "Northen Russians",
  "Алтайский": "Northen Russians",
  "Смоленская": "Northen Russians",
  "Ханты-Мансийский Автономный округ - Югра": "Northen Russians",
  "Нижегородская": "Northen Russians",
  "Удмуртская": "Northen Russians",
  "Калужская": "Northen Russians",
  "Севастополь": "Ukranians",
  "Витебская": "Belarusians",
  "Харьковская": "Ukranians",
  "Кемеровская область - Кузбасс": "Northen Russians",
  "Волгоградская": "Southern Russians",
  "Ташкент": "Unknown",
  "Ленинградская": "Northen Russians",
  "Курганская": "Northen Russians",
  "Ивановская": "Northen Russians",
  "Киевская": "Ukranians",
  "Thuringia": "Unknown",
  "Ивано-Франковская": "Ukranians",
  "Dushanbe": "Unknown",
  "Амурская": "Northen Russians",
  "Yerevan": "Unknown",
  "Ненецкий": "Northen Russians",
  "Ulaanbaatar": "Unknown",
  "Черниговская": "Ukranians",
  "Костромская": "Northen Russians",
  "Северная Осетия - Алания": "Southern Russians",
  "Новгородская": "Northen Russians",
  "T'bilisi": "Unknown",
  "Закарпатская": "Ukranians",
  "Гомельская": "Belarusians",
  "Бурятия": "Northen Russians",
  "Tatarstan": "Northen Russians",
  "Имеретия": "Unknown",
  "Белгородская": "Northen Russians",
  "Астраханская": "Southern Russians"
}


In [6]:
d3={
  "Тюменская": "Northen Russians",
  "Gagauzia": "Unknown",
  "Ошская": "Unknown",
  "Брянская": "Northen Russians",
  "Черновицкая": "Ukranians",
  "Ставропольский": "Southern Russians",
  "Zaporozhskaya": "Ukranians",
  "Kyiv": "Ukranians",
  "Casablanca-Settat": "Unknown",
  "Cherkasskaya": "Ukranians",
  "Абхазия": "Unknown",
  "Самцхе-Джавахети": "Unknown",
  "Приморский": "Northen Russians",
  "Волынская": "Ukranians",
  "Минск": "Belarusians",
  "Воронежская": "Northen Russians",
  "Житомирская": "Ukranians",
  "Еврейская": "Northen Russians",
  "Акмолинская": "Unknown",
  "Шида-Картли": "Unknown",
  "Sumqayit": "Unknown",
  "Ферганская": "Unknown",
  "Иссык-Кульская": "Unknown",
  "Бишкек": "Unknown",
  "Карагандинская": "Unknown",
  "Рига": "Unknown",
  "Магаданская": "Northen Russians",
  "Киев": "Ukranians",
  "Мангистауская": "Unknown",
  "Сахалинская": "Northen Russians",
  "Брестская": "Belarusians",
  "Камчатский": "Northen Russians",
  "Fergana": "Unknown",
  "Марий Эл": "Northen Russians",
  "Орловская": "Northen Russians",
  "Чеченская": "Southern Russians",
  "Среднечешский край": "Unknown",
  "Gadabay Rayon": "Unknown",
  "Кировоградская": "Ukranians",
  "Карачаево-Черкесская": "Southern Russians",
  "Surxondaryo": "Unknown",
  "Владимирская": "Northen Russians",
  "Нью-Йорк": "Unknown",
  "Бургенланд": "Unknown",
  "Шлезвиг-Гольштейн": "Unknown",
  "Луганская": "Ukranians",
  "Республика Крым": "Ukranians"
}


In [7]:
d4={
  "Algiers": "Unknown",
  "Mazovia": "Unknown",
  "Бранденбург": "Unknown",
  "Нур-Султан": "Unknown",
  "Алматы": "Unknown",
  "Винницкая": "Ukranians",
  "Ровненская": "Ukranians",
  "Западно-Казахстанская": "Unknown",
  "Томская": "Northen Russians",
  "Permsky": "Northen Russians",
  "Minskaya": "Belarusians",
  "Хакасия": "Northen Russians",
  "Чувашская республика": "Northen Russians",
  "Могилёвская": "Belarusians",
  "Оргеевский": "Unknown",
  "Calarasi": "Unknown",
  "Новоаненский": "Unknown",
  "Samarqand": "Unknown",
  "Одесская": "Ukranians",
  "Klaipeda County": "Unknown",
  "Vilnius": "Unknown",
  "Stavropolsky": "Southern Russians",
  "Хмельницкая": "Ukranians",
  "Подляское воеводство": "Unknown",
  "Toshkent": "Unknown",
  "Бавария": "Unknown",
  "Саха /Якутия/": "Northen Russians",
  "Мазовецкое воеводство": "Unknown",
  "Берлин": "Unknown",
  "Shaki City": "Unknown",
  "Окницкий": "Unknown",
  "Chechenskaya": "Southern Russians",
  "Ширакская": "Unknown",
  "Кабардино-Балкарская": "Southern Russians",
  "Атырауская": "Unknown",
  "Чуйская": "Unknown",
  "Северо-Казахстанская": "Unknown",
  "Николаевская": "Ukranians",
  "Флорештский": "Unknown",
  "Мурманская": "Northen Russians",
  "Siauliai": "Unknown",
  "Костанайская": "Unknown",
  "Карелия": "Northen Russians",
  "Ингушетия": "Southern Russians",
  "Адыгея": "Southern Russians",
  "Xacmaz": "Unknown",
  "Наманганская": "Unknown"
}


In [8]:
d5={
  "Гродненская": "Belarusians",
  "Transnistria": "Unknown",
  "Сорокский": "Unknown",
  "Saint-Petersburg": "Northen Russians",
  "Saxony": "Unknown",
  "Quba": "Unknown",
  "Алтай": "Northen Russians",
  "Актюбинская": "Unknown",
  "Саксония-Анхальт": "Unknown",
  "Navoiy": "Unknown",
  "Свентокшишское воеводство": "Unknown",
  "Mecklenburg-Vorpommern": "Unknown",
  "Алматинская": "Unknown",
  "Муш": "Unknown",
  "Тернопольская": "Ukranians",
  "Яс-Надькун-Сольнок": "Unknown",
  "Iasi": "Unknown",
  "Марыйский велаят": "Unknown",
  "Автономная Республика Аджария": "Unknown",
  "Гродненская область": "Belarusians",
  "Лорийская": "Unknown",
  "Жамбылская": "Unknown",
  "Абайская": "Unknown",
  "Utena": "Unknown",
  "Gorod Sevastopol": "Ukranians",
  "Шымкент": "Unknown",
  "Хайду-Бихар": "Unknown",
  "Samegrelo and Zemo Svaneti": "Unknown",
  "Basra": "Unknown",
  "Тараклийский": "Unknown",
  "Туркестанская": "Unknown",
  "Montana": "Unknown",
  "Toshkent Shahri": "Unknown",
  "Ида-Вирумаа": "Unknown",
  "Донецкая Народная": "Ukranians",
  "Нижнесилезское воеводство": "Unknown",
  "Luganskaya Narodnaya": "Ukranians",
  "Kaunas": "Unknown",
  "Луганская Народная": "Ukranians",
  "Siazan Rayon": "Unknown",
  "Республика Каракалпакстан": "Unknown"
}


In [9]:
d6={
  "Xankandi Sahari": "Unknown",
  "Khatlon": "Unknown",
  "Приморская": "Northen Russians",
  "Тартумаа": "Unknown",
  "Калмыкия": "Southern Russians",
  "Лебапский велаят": "Unknown",
  "Minnesota": "Unknown",
  "Нью-Джерси": "Unknown",
  "Самаркандская": "Unknown",
  "Рышканский": "Unknown",
  "Selenge": "Unknown",
  "Бухара": "Unknown",
  "Кишинёв": "Unknown",
  "Chisinau Municipality": "Unknown",
  "Уусимаа": "Unknown",
  "Ucar": "Unknown",
  "Тбилиси": "Unknown",
  "Квемо-Картли": "Unknown",
  "Sirdaryo Region": "Unknown",
  "Balti": "Unknown",
  "Джалал-Абадская": "Unknown",
  "Rezekne": "Unknown",
  "Физулинский": "Unknown",
  "Ереван": "Unknown",
  "Krasnodarsky": "Southern Russians",
  "Ташкентская": "Unknown",
  "Тавушская": "Unknown",
  "Могилёвская область": "Belarusians",
  "Согдийская": "Unknown",
  "Баткенская": "Unknown",
  "Vilanu": "Unknown",
  "Нарынская": "Unknown",
  "Англия": "Unknown",
  "Утрехт": "Unknown",
  "Центральная Сербия": "Unknown",
  "Гомельская область": "Belarusians",
  "Байконур": "Unknown",
  "Шолданештский": "Unknown",
  "Бендеры": "Unknown",
  "Любуское воеводство": "Unknown",
  "Jalilabad": "Unknown",
  "Арагацотнская": "Unknown",
  "Чукотский": "Northen Russians",
  "Кашкадаринская": "Unknown",
  "Андижанская": "Unknown",
  "Ашхабад": "Unknown",
  "Даугавпилс": "Unknown"
}


In [10]:
d7={
  "Кашкадарьинская": "Unknown",
  "Port Louis": "Unknown",
  "Мцхета-Мтианети": "Unknown",
  "Унгенский": "Unknown",
  "Mordoviya": "Northen Russians",
  "Орду": "Unknown",
  "Jakarta": "Unknown",
  "Нахичевань": "Unknown",
  "Балканский велаят": "Unknown",
  "Dasoguz": "Unknown",
  "Вена": "Unknown",
  "Тыва": "Northen Russians",
  "Гехаркуникская": "Unknown",
  "Баден-Вюртемберг": "Unknown",
  "Dashkasan Rayon": "Unknown",
  "Shusha": "Unknown",
  "Араратская": "Unknown",
  "Kars": "Unknown",
  "Gumushane": "Unknown",
  "Olomoucky": "Unknown",
  "Bacau": "Unknown",
  "Харьюмаа": "Unknown",
  "Анталья": "Unknown",
  "Inner Mongolia": "Unknown",
  "Пенсильвания": "Unknown",
  "Калифорния": "Unknown",
  "Кахетия": "Unknown",
  "Osaka": "Unknown",
  "Kabul": "Unknown",
  "Сырдарьинская": "Unknown",
  "Герат": "Unknown",
  "Hlavni mesto Praha": "Unknown",
  "Ganja City": "Unknown",
  "Delhi": "Unknown",
  "Кызылординская": "Unknown",
  "Budapest": "Unknown",
  "Sofia-Capital": "Unknown",
  "Западнопоморское воеводство": "Unknown",
  "Kumanovo": "Unknown",
  "Smolyan": "Unknown",
  "Liaoning": "Unknown",
  "Окаяма": "Unknown",
  "Sao Paulo": "Unknown",
  "Rezekne Municipality": "Unknown",
  "Даугавпилсский край": "Unknown",
  "Littoral": "Unknown",
  "Пярнумаа": "Unknown",
  "Висконсин": "Unknown",
  "Cajamarca": "Unknown"
}


In [11]:
d8={
  "Пьемонт": "Unknown",
  "Прованс — Альпы — Лазурный Берег": "Unknown",
  "Лигурия": "Unknown",
  "Малопольское воеводство": "Unknown",
  "Битлис": "Unknown",
  "Суворовский": "Southern Russians", 
  "Онтарио": "Unknown",
  "Arges": "Unknown",
  "Флорида": "Unknown",
  "Йозгат": "Unknown",
  "Beijing": "Unknown",
  "Zaqatala": "Unknown",
  "Лиепая": "Unknown",
  "Samarskaya": "Northen Russians",
  "Хошимин": "Unknown",
  "Нахичеванская Автономная Республика": "Unknown",
  "Джизакская": "Unknown",
  "Гессен": "Unknown",
  "Лацио": "Unknown",
  "Новый Южный Уэльс": "Unknown",
  "Heilongjiang": "Unknown",
  "Marijampole County": "Unknown",
  "Baghdad": "Unknown",
  "Sistan and Baluchestan": "Unknown",
  "Северная Голландия": "Unknown",
  "Таласская": "Unknown",
  "Xorazm": "Unknown",
  "Ахалский велаят": "Unknown",
  "Армавирская": "Southern Russians",
  "Bender Municipality": "Unknown",
  "Malpils": "Unknown",
  "Banskobystricky kraj": "Unknown",
  "Нижняя Австрия": "Unknown",
  "Centre": "Unknown",
  "Виктория": "Unknown",
  "Центральный": "Northen Russians",
  "Jerusalem": "Unknown",
  "Массачусетс": "Unknown",
  "Valka": "Unknown",
  "Елгава": "Unknown",
  "Районы республиканского подчинения": "Unknown",
  "Yevlax City": "Unknown",
  "Aizpute": "Unknown",
  "Ван": "Unknown"
}


In [12]:
d9={
  "Кагульский": "Unknown",
  "Qabala Rayon": "Unknown",
  "Ленстер": "Unknown",
  "Bucuresti": "Unknown",
  "Lombardy": "Unknown",
  "Сюникская": "Unknown",
  "Котайкская": "Unknown",
  "Синд": "Unknown",
  "Опольское воеводство": "Unknown",
  "Addis Ababa": "Unknown",
  "Вентспилс": "Unknown",
  "Нижняя Саксония": "Unknown",
  "Presovsky kraj": "Unknown",
  "Фейер": "Unknown",
  "Causeni": "Unknown",
  "City of Zagreb": "Unknown",
  "Zagrebacka": "Unknown",
  "Koprivnicko-Krizevacka": "Unknown",
  "Slavonski Brod-Posavina": "Unknown",
  "Силезское воеводство": "Unknown",
  "Женева": "Unknown",
  "Астана": "Unknown",
  "Техас": "Unknown",
  "Пара": "Unknown",
  "Ludzas novads": "Unknown",
  "Самегрело — Земо-Сванети": "Unknown",
  "Izmir": "Unknown",
  "Астурия": "Unknown",
  "Коджаэли": "Unknown",
  "Жетысуская": "Unknown",
  "Dimashq": "Unknown",
  "Central Macedonia": "Unknown",
  "Bursa": "Unknown",
  "Кютахья": "Unknown",
  "Стамбул": "Unknown",
  "Кастамону": "Unknown",
  "Fuezuli": "Unknown",
  "Страшенский": "Unknown",
  "Бретань": "Unknown",
  "Биело-Поле": "Unknown",
  "Pljevlja": "Unknown",
  "Рача-Лечхуми и Нижняя Сванетия": "Unknown",
  "Imisli": "Unknown",
  "Federation of B&H": "Unknown",
  "Tashkent": "Unknown",
  "Vitebskaya": "Belarusians"
}


In [13]:
d10={
  "Бремен": "Unknown",
  "Goeycay": "Unknown",
  "Южный": "Southern Russians",
  "Panama": "Unknown",
  "Buenos Aires F.D.": "Unknown",
  "Бухарская": "Unknown",
  "Khojavend": "Unknown",
  "Западная Бенгалия": "Unknown",
  "Mexico City": "Unknown",
  "Jalisco": "Unknown",
  "Tartar Rayon": "Unknown",
  "Tovuz": "Unknown",
  "Sharqia": "Unknown",
  "East Azerbaijan": "Unknown",
  "Габалинский": "Unknown",
  "Krasnoyarsky": "Northen Russians",
  "Karagandinskaya": "Unknown",
  "Поморское воеводство": "Unknown",
  "Balakan Rayon": "Unknown",
  "Panevezys": "Unknown",
  "Вашингтон": "Unknown",
  "Алабама": "Unknown",
  "Бач-Кишкун": "Unknown",
  "Гагаузия": "Unknown",
  "Stockholm": "Unknown",
  "Южное Саво": "Unknown",
  "Куско": "Unknown",
  "Michoacan": "Unknown",
  "Иллинойс": "Unknown",
  "Лиссабон": "Unknown",
  "Analamanga": "Unknown",
  "Agdam": "Unknown",
  "Moskovskaya": "Northen Russians",
  "Tel Aviv": "Unknown",
  "Саскачеван": "Unknown",
  "Единецкий": "Unknown",
  "Область Мадрид": "Unknown",
  "Saratovskaya": "Northen Russians",
  "Goygol Rayon": "Unknown",
  "Ilukste": "Unknown",
  "Мугла": "Unknown",
  "Vojvodina": "Unknown",
  "Opstina Pluzine": "Unknown",
  "Мэриленд": "Unknown",
  "Сынжерейский": "Unknown",
  "Варминско-Мазурское воеводство": "Unknown",
  "Donetskaya": "Ukranians"
}


In [14]:
d11={
  "Dnepropetrovskaya": "Ukranians",
  "Preilu novads": "Unknown",
  "Коннектикут": "Unknown",
  "Kareliya": "Northen Russians",
  "Ile-de-France": "Unknown",
  "Xinjiang": "Unknown",
  "Samaxi": "Unknown",
  "Глодянский": "Unknown",
  "Zanzibar Urban/West": "Unknown",
  "Prague": "Unknown",
  "Penzenskaya": "Northen Russians",
  "Veliko Tarnovo": "Unknown",
  "Tripoli": "Unknown",
  "Masally": "Unknown",
  "Bratislavsky Kraj": "Unknown",
  "Abseron": "Unknown",
  "Arkhangelskaya": "Northen Russians",
  "San Jose": "Unknown",
  "Федеральный округ": "Northen Russians",
  "Terengganu": "Unknown",
  "Hwanghae-namdo": "Unknown",
  "Kuldigas novads": "Unknown",
  "Agdas": "Unknown",
  "Kalbajar": "Unknown",
  "Irkutskaya": "Northen Russians",
  "Voronezhskaya": "Northen Russians",
  "Ляэне-Вирумаа": "Unknown",
  "Гурия": "Unknown",
  "Теленештский": "Unknown",
  "Pyongyang": "Unknown",
  "Telsiai": "Unknown",
  "Tverskaya": "Northen Russians",
  "Улытауская": "Unknown",
  "South Kurzeme Municipality": "Unknown",
  "Хайфский округ": "Unknown",
  "West Bank": "Unknown",
  "Orlovskaya": "Northen Russians",
  "Лодзинское воеводство": "Unknown",
  "Pazardzhik": "Unknown",
  "Жиззахская": "Unknown",
  "Yaroslavskaya": "Northen Russians",
  "Smolenskaya": "Northen Russians",
  "Балх": "Unknown",
  "Люблинское воеводство": "Unknown"
}


In [15]:
d12={
  "Liepāja": "Unknown",
  "Kemerovskaya oblast - Kuzbass": "Northen Russians",
  "Songkhla": "Unknown",
  "Qubadli": "Unknown",
  "Лерикский": "Unknown",
  "Parana": "Unknown",
  "West Azerbaijan": "Unknown",
  "Миссури": "Unknown",
  "Окленд": "Unknown",
  "Qobustan": "Unknown",
  "Komi": "Northen Russians",
  "Миннесота": "Unknown",
  "Lankaran": "Unknown",
  "Ляэнемаа": "Unknown",
  "Gorno-Badakhshan": "Unknown",
  "Ardabil": "Unknown",
  "Havana": "Unknown",
  "Limassol": "Unknown",
  "Troms og Finnmark": "Unknown",
  "Округ Колумбия": "Unknown",
  "Nouvelle-Aquitaine": "Unknown",
  "Tartu": "Unknown",
  "Grodnenskaya": "Belarusians",
  "Alytus": "Unknown",
  "Sabirabad": "Unknown",
  "Taurage County": "Unknown",
  "Beyrouth": "Unknown",
  "Сурхандарьинская": "Unknown",
  "Jurmala": "Unknown",
  "Osun": "Unknown",
  "Valmiera": "Unknown",
  "Qusar": "Unknown",
  "Bauska Municipality": "Unknown",
  "Kraslavas novads": "Unknown",
  "Екабпилсский край": "Unknown",
  "Kralovehradecky kraj": "Unknown",
  "Brussels Capital": "Unknown",
  "Varna": "Unknown",
  "Бричанский": "Unknown",
  "Резинский": "Unknown",
  "Agri": "Unknown",
  "Tehran": "Unknown",
  "Jiangsu": "Unknown",
  "Vysocina": "Unknown",
  "Yambol": "Unknown",
  "West Greece": "Unknown",
  "Тоскана": "Unknown",
  "Karlovacka": "Unknown"
}


In [16]:
d13={
  "Chungcheongnam-do": "Unknown",
  "Леовский": "Unknown",
  "Западная Ява": "Unknown",
  "Вырумаа": "Unknown",
  "Административно-территориальные единицы левобережья Днестра": "Unknown",
  "Айдахо": "Unknown",
  "Sigulda": "Unknown",
  "Трабзон": "Unknown",
  "South Aegean": "Unknown",
  "Ankara": "Unknown",
  "Йыгевамаа": "Unknown",
  "Salaspils": "Unknown",
  "Kharkovskaya": "Ukranians",
  "Occitanie": "Unknown",
  "Эскишехир": "Unknown",
  "Южная Голландия": "Unknown",
  "Вайоцдзорская": "Unknown",
  "Северный Рейн-Вестфалия": "Unknown",
  "Leningradskaya": "Northen Russians",
  "Тироль": "Unknown",
  "Малатья": "Unknown",
  "Haskovo": "Unknown",
  "Vaud": "Unknown",
  "Luganskaya": "Ukranians",
  "Buryatiya": "Northen Russians",
  "Akmolinskaya": "Unknown",
  "Ваш": "Unknown",
  "Chelyabinskaya": "Northen Russians",
  "Zakarpatskaya": "Ukranians",
  "Novosibirskaya": "Northen Russians",
  "Yevlax": "Unknown",
  "Македонска-Каменица": "Unknown",
  "Bauskas Rajons": "Unknown",
  "Celje": "Unknown",
  "Xizi": "Unknown",
  "Aluksnes Novads": "Unknown",
  "Rezeknes Novads": "Unknown",
  "Вильяндимаа": "Unknown",
  "Rio de Janeiro": "Unknown",
  "Cairo": "Unknown",
  "Tianjin": "Unknown",
  "Минас-Жерайс": "Unknown",
  "Ustecky kraj": "Unknown",
  "Британская Колумбия": "Unknown"
}


In [17]:
d14={
  "Kekava": "Unknown",
  "Ogre": "Unknown",
  "Валгамаа": "Unknown",
  "Izmir Province": "Unknown",
  "South Karelia": "Unknown",
  "Vostochno-Kazakhstanskaya": "Unknown",
  "Даларна": "Unknown",
  "Southern Region": "Unknown",
  "Апулия": "Unknown",
  "Саксония": "Unknown",
  "Рейнланд-Пфальц": "Unknown",
  "Aktyubinskaya": "Unknown",
  "Ryazanskaya": "Northen Russians",
  "Il-Gzira": "Unknown",
  "Ардахан": "Unknown",
  "Тель-Авивский округ": "Unknown",
  "Phnom Penh": "Unknown",
  "Kabardino-Balkarskaya": "Southern Russians",
  "Кампания": "Unknown",
  "Busan": "Unknown",
  "Augsdaugava Municipality": "Unknown",
  "Dubai": "Unknown",
  "Shanghai": "Unknown",
  "Shida Kartli": "Unknown",
  "Barda": "Unknown",
  "Невшатель": "Unknown",
  "Род-Айленд": "Unknown",
  "Телангана": "Unknown",
  "Quebec": "Unknown",
  "Sakhalinskaya": "Northen Russians",
  "Evreyskaya": "Northen Russians",
  "Hentiy": "Unknown",
  "Mogilevskaya": "Belarusians",
  "Gao": "Unknown",
  "Варсинайс-Суоми": "Unknown",
  "Навоийская": "Unknown",
  "Гусарский": "Unknown",
  "Tukuma novads": "Unknown",
  "Подкарпатское воеводство": "Unknown",
  "Primorsky": "Northen Russians",
  "Калабрия": "Unknown",
  "Nevsehir": "Unknown",
  "Kostromskaya": "Northen Russians",
  "Rabat-Sale-Kenitra": "Unknown",
  "Куявско-Поморское воеводство": "Unknown"
}


In [18]:
d15={
  "Northern District": "Unknown",
  "Waikato": "Unknown",
  "Чимишлийский": "Unknown",
  "Guangdong": "Unknown",
  "Haciqabul": "Unknown",
  "Flanders": "Unknown",
  "Las Tunas": "Unknown",
  "Mingacevir City": "Unknown",
  "Daugavpils": "Unknown",
  "Jēkabpils Municipality": "Unknown",
  "Lusaka": "Unknown",
  "East Gobi Aymag": "Unknown",
  "Burgas": "Unknown",
  "Тасмания": "Unknown",
  "Zhitomirskaya": "Ukranians",
  "Pinar del Rio": "Unknown",
  "Хатай": "Unknown",
  "Харьяна": "Unknown",
  "Zabaykalsky": "Northen Russians",
  "Юта": "Unknown",
  "Darhan Uul": "Unknown",
  "Дрокиевский": "Unknown",
  "Фалештский": "Unknown",
  "Хорасан-Резави": "Unknown",
  "Сааремаа": "Unknown",
  "Lima": "Unknown",
  "Jilin": "Unknown",
  "Jelgava Municipality": "Unknown",
  "Khakasiya": "Northen Russians",
  "East Macedonia and Thrace": "Unknown",
  "Евлеборг": "Unknown",
  "Tomskaya": "Northen Russians",
  "Baladiyat ad Dawhah": "Unknown",
  "Stara Zagora": "Unknown",
  "Амасонас": "Unknown",
  "Пьюра": "Unknown",
  "Хынчештский": "Unknown",
  "Словенска-Бистрица": "Unknown",
  "Surat Thani": "Unknown",
  "Madona Municipality": "Unknown",
  "Belgorodskaya": "Northen Russians",
  "Каталония": "Unknown",
  "Pskovskaya": "Northen Russians",
  "Мичиган": "Unknown",
  "Antioquia": "Unknown",
  "Grand Est": "Unknown",
  "Zurich": "Unknown"
}


In [19]:
d16={
  "Pichincha": "Unknown",
  "Hanoi": "Unknown",
  "Центральная Финляндия": "Unknown",
  "Volgogradskaya": "Southern Russians",
  "Tashkentskaya": "Unknown",
  "Holguin": "Unknown",
  "Tunis Governorate": "Unknown",
  "Пяйят-Хяме": "Unknown",
  "Хатлонская": "Unknown",
  "Tokyo": "Unknown",
  "Тунис": "Unknown",
  "Иль-де-Франс": "Unknown",
  "Османие": "Unknown",
  "Сан-Паулу": "Unknown",
  "Karlovarsky kraj": "Unknown",
  "Zilinsky kraj": "Unknown",
  "Андалусия": "Unknown",
  "Газни": "Unknown",
  "Bali": "Unknown",
  "Бессарабский": "Unknown",
  "Канзас": "Unknown",
  "Гоа": "Unknown",
  "Ammochostos": "Unknown",
  "Attica": "Unknown",
  "Hamburg": "Unknown",
  "Умбрия": "Unknown",
  "Cesu Rajons": "Unknown",unique_node_ibd_ids
  "Moravskoslezsky": "Unknown",
  "Канагава": "Unknown",
  "Balvu Novads": "Unknown",
  "Harare": "Unknown",
  "Душанбе": "Unknown",
  "Chon Buri": "Unknown",
  "Фукуи": "Unknown",
  "Улутауская": "Unknown",
  "Stolichnaya": "Unknown",
  "Valparaiso": "Unknown",
  "Maule Region": "Unknown",
  "Mecca Region": "Unknown",
  "Murmanskaya": "Northen Russians",
  "Phuket": "Unknown",
  "Sumskaya": "Ukranians",
  "Kurskaya": "Northen Russians",
  "Тичино": "Unknown",
  "Shirvan": "Unknown",
  "Дондюшанский": "Unknown",
  "Ba Ria-Vung Tau": "Unknown",
  "Kirovskaya": "Northen Russians",
  "Sverdlovskaya": "Northen Russians",
  "Rostovskaya": "Southern Russians"
}


In [20]:
d17={
  "Baranya": "Unknown",
  "Dennery": "Unknown",
  "Xocali": "Unknown",
  "Ярвамаа": "Unknown",
  "Salyan": "Unknown",
  "Tungurahua": "Unknown",
  "Tulskaya": "Northen Russians",
  "Khersonskaya": "Ukranians",
  "Nicosia": "Unknown",
  "Кентербери": "Unknown",
  "Nam Dinh": "Unknown",
  "Diyarbakir": "Unknown",
  "Kocaeli": "Unknown",
  "Эрзурум": "Unknown",
  "Colorado": "Unknown",
  "Керетаро": "Unknown",
  "Ниспоренский": "Unknown",
  "Qazvin": "Unknown",
  "Базиликата": "Unknown",
  "Los Lagos Region": "Unknown",
  "Aysen": "Unknown",
  "Chubut": "Unknown",
  "Vologodskaya": "Northen Russians",
  "Goranboy": "Unknown",
  "Bryanskaya": "Northen Russians",
  "Байконыр Г.А.": "Unknown",
  "Rogaland": "Unknown",
  "Галисия": "Unknown",
  "Область Валенсия": "Unknown",
  "Артвин": "Unknown",
  "Ismayilli": "Unknown",
  "Santiago": "Unknown",
  "Querétaro City": "Unknown",
  "Antofagasta": "Unknown",
  "Poltavskaya": "Ukranians",
  "Sevastopol": "Ukranians",
  "South Sinai": "Unknown",
  "Зала": "Unknown",
  "Almaty": "Unknown",
  "Oguz": "Unknown",
  "Уэльс": "Unknown",
  "Berane": "Unknown",
  "Gusinje": "Unknown",
  "Srpska": "Unknown",
  "Bishkek": "Unknown",
  "Дренте": "Unknown",
  "Krym": "Ukranians",
  "Chisinau": "Unknown",
  "Nizhegorodskaya": "Northen Russians",
  "Coquimbo Region": "Unknown",
  "Crimea": "Ukranians",
  "Vinnitskaya": "Ukranians",
  "Oslo": "Unknown",
  "Olaine": "Unknown"
}


In [21]:
d18={
  "Nueva Esparta": "Unknown",
  "Carabobo": "Unknown",
  "Саар": "Unknown",
  "Capital Region": "Unknown",
  "Kakheti": "Unknown",
  "Galati": "Unknown",
  "Lipetskaya": "Northen Russians",
  "Biobio": "Unknown",
  "Central Serbia": "Unknown",
  "Вестра-Гёталанд": "Unknown",
  "Neftcala": "Unknown",
  "Венеция": "Unknown",
  "Severnaya Osetiya - Alaniya": "Southern Russians",
  "Dobeles novads": "Unknown",
  "Yardimli": "Unknown",
  "Larnaka": "Unknown",
  "Altaysky": "Northen Russians",
  "Pavlodarskaya": "Unknown",
  "Герцег-Нови": "Unknown",
  "Gomelskaya": "Belarusians",
  "Saatli": "Unknown",
  "Orenburgskaya": "Northen Russians",
  "Вильнюсский уезд": "Unknown",
  "Кантемирский": "Unknown",
  "Lerik": "Unknown",
  "Кюменлааксо": "Unknown",
  "Каунасский уезд": "Unknown",
  "Яловенский": "Unknown",
  "Синоп": "Unknown",
  "Western Cape": "Unknown",
  "Cienfuegos Province": "Unknown",
  "Kvemo Kartli": "Unknown",
  "Северная Остроботния": "Unknown",
  "Basel-Landschaft": "Unknown",
  "Auvergne-Rhone-Alpes": "Unknown",
  "Quintana Roo": "Unknown",
  "Пылвамаа": "Unknown",
  "Constanta": "Unknown",
  "Криулянский": "Unknown",
  "Singida": "Unknown",
  "Kalmykiya": "Southern Russians",
  "Takeo": "Unknown",
  "Pristina": "Unknown",
  "Bitola": "Unknown",
  "Kichevo": "Unknown"
}


In [22]:
d19={
  "Tambovskaya": "Northen Russians",
  "Chernigovskaya": "Ukranians",
  "Страна Басков": "Unknown",
  "Udmurtskaya": "Northen Russians",
  "Комаром-Эстергом": "Unknown",
  "Огайо": "Unknown",
  "Бихар": "Unknown",
  "Цесисский край": "Unknown",
  "Smiltene": "Unknown",
  "Kurdamir Rayon": "Unknown",
  "Tyumenskaya": "Northen Russians",
  "Bilasuvar Rayon": "Unknown",
  "Штирия": "Unknown",
  "West": "Unknown",
  "Dzhizakskaya": "Unknown",
  "Марке": "Unknown",
  "Guemueshane Province": "Unknown",
  "Bashkortostan": "Northen Russians",
  "Южная Австралия": "Unknown",
  "Сатакунта": "Unknown",
  "Grad Skopje": "Unknown",
  "Beylaqan": "Unknown",
  "Гавайи": "Unknown",
  "Karachaevo-Cherkesskaya": "Southern Russians",
  "Talsi Municipality": "Unknown",
  "Уамбо": "Unknown",
  "Agsu": "Unknown",
  "California": "Unknown",
  "Orhon": "Unknown",
  "Kyustendil": "Unknown",
  "West Virginia": "Unknown",
  "Faro": "Unknown",
  "Южноморавский край": "Unknown",
  "Кайсери": "Unknown",
  "Emilia-Romagna": "Unknown",
  "Abkhazia": "Unknown",
  "Andalusia": "Unknown",
  "Adygeya": "Southern Russians",
  "Kaliningradskaya": "Northen Russians",
  "Dzhalal-Abadskaya": "Unknown",
  "Qax": "Unknown",
  "Massachusetts": "Unknown",
  "Amurskaya": "Northen Russians",
  "Ulyanovskaya": "Northen Russians",
  "Mary El": "Northen Russians"
}


In [23]:
d20={
  "Minsk": "Belarusians",
  "Hauts-de-France": "Unknown",
  "County of Zadar": "Unknown",
  "Котор": "Unknown",
  "Judea and Samaria Area": "Unknown",
  "Кингстон": "Unknown",
  "Kars Province": "Unknown",
  "Hesse": "Unknown",
  "Suceava": "Unknown",
  "Нью-Мехико": "Unknown",
  "Region of Magallanes": "Unknown",
  "Nuble": "Unknown",
  "Bern": "Unknown",
  "Liberecky kraj": "Unknown",
  "Brestskaya": "Belarusians",
  "Florida": "Unknown",
  "Normandy": "Unknown",
  "Ludza Municipality": "Unknown",
  "Medina Region": "Unknown",
  "England": "Unknown",
  "Карс": "Unknown",
  "Хорезмская": "Unknown",
  "Aizkraukle Municipality": "Unknown",
  "Вермонт": "Unknown",
  "Nograd": "Unknown",
  "Dhaka": "Unknown",
  "Tukums Municipality": "Unknown",
  "Varaklani Municipality": "Unknown",
  "Кайнуу": "Unknown",
  "Aluksne Municipality": "Unknown",
  "Западная Австралия": "Unknown",
  "Vladimirskaya": "Northen Russians",
  "New Hampshire": "Unknown",
  "Luanda": "Unknown",
  "Bengo": "Unknown",
  "Fujian": "Unknown",
  "Plovdiv": "Unknown",
  "East Kalimantan": "Unknown",
  "Abayskaya": "Unknown",
  "Зеландия": "Unknown",
  "Vestfold og Telemark": "Unknown",
  "North Denmark": "Unknown",
  "Джорджия": "Unknown",
  "Раджастхан": "Unknown"
}


In [24]:
d21={
  "Khabarovsky": "Northen Russians",
  "Shaki": "Unknown",
  "Cordoba": "Unknown",
  "Livani": "Unknown",
  "Bagmati Province": "Unknown",
  "Пенджаб": "Unknown",
  "Kievskaya": "Ukranians",
  "Kaluzhskaya": "Northen Russians",
  "La Libertad": "Unknown",
  "Callao": "Unknown",
  "Arequipa": "Unknown",
  "Cojedes": "Unknown",
  "Golestan": "Unknown",
  "Хийумаа": "Unknown",
  "Пирканмаа": "Unknown",
  "Зардобский": "Unknown",
  "Великопольское воеводство": "Unknown",
  "Astrakhanskaya": "Southern Russians",
  "Smiltene Municipality": "Unknown",
  "Santiago de Cuba": "Unknown",
  "Luxembourg": "Unknown",
  "Lisbon": "Unknown",
  "Santarem": "Unknown",
  "East Aimak": "Unknown",
  "Dagestan": "Southern Russians",
  "Kaunassky uezd": "Unknown",
  "Villa Clara Province": "Unknown",
  "Queensland": "Unknown",
  "Los Rios Region": "Unknown",
  "Khanty-Mansiysky Avtonomny okrug - Yugra": "Northen Russians",
  "Tulcea": "Unknown",
  "Алитусский уезд": "Unknown",
  "Делавэр": "Unknown",
  "Сицилия": "Unknown",
  "Johor": "Unknown",
  "Aden": "Unknown",
  "Ica": "Unknown",
  "Ancash": "Unknown",
  "Shabran": "Unknown",
  "Shumen": "Unknown",
  "Шабранский": "Unknown",
  "Рапламаа": "Unknown",
  "Guatemala": "Unknown",
  "Araucania": "Unknown",
  "Puebla": "Unknown",
  "Guanajuato": "Unknown"
}


In [25]:
d22={
  "Ибараки": "Unknown",
  "Ida-Virumaa": "Unknown",
  "Aghjabadi Rayon": "Unknown",
  "Лимасол": "Unknown",
  "Brittany Region": "Unknown",
  "Praia": "Unknown",
  "Хубэй": "Unknown",
  "Орегон": "Unknown",
  "Khmelnitskaya": "Ukranians",
  "Omskaya": "Northen Russians",
  "Rovnenskaya": "Ukranians",
  "Паневежский уезд": "Unknown",
  "Северное Саво": "Unknown",
  "Заире": "Unknown",
  "Pleven": "Unknown",
  "Odesskaya": "Ukranians",
  "Dobele Municipality": "Unknown",
  "Cesis Municipality": "Unknown",
  "Balvi Municipality": "Unknown",
  "Kuldiga Municipality": "Unknown",
  "Seoul": "Unknown",
  "Brasov": "Unknown",
  "Podgorica": "Unknown",
  "Клайпедский уезд": "Unknown",
  "Мухафаза Тартус": "Unknown",
  "Канарские Острова": "Unknown",
  "Atacama": "Unknown",
  "Riga": "Unknown",
  "Мухафаза Халеб": "Unknown",
  "Uusimaa": "Unknown",
  "North Savo": "Unknown",
  "Kymenlaakso": "Unknown",
  "Kainuunkyla": "Unknown",
  "Southwest Finland": "Unknown",
  "Больцано": "Unknown",
  "Harju": "Unknown",
  "Jerash": "Unknown",
  "Ljubljana": "Unknown",
  "Южный Ливан": "Unknown",
  "San Pedro": "Unknown",
  "Suez": "Unknown",
  "Huancavelica": "Unknown",
  "Salta": "Unknown",
  "Аджария": "Unknown",
  "Верхняя Австрия": "Unknown",
  "Chuvashskaya respublika": "Northen Russians"
}


In [26]:
d23={
  "Ыгдыр": "Unknown",
  "Altay": "Northen Russians",
  "Шэньси": "Unknown",
  "Окинава": "Unknown",
  "Jelgava": "Unknown",
  "Mexico": "Unknown",
  "Coahuila": "Unknown",
  "Sinaloa": "Unknown",
  "Колорадо": "Unknown",
  "Loreto": "Unknown",
  "Amanat Alasimah": "Unknown",
  "Lvovskaya": "Ukranians",
  "Констанца": "Unknown",
  "Мухафаза Латакия": "Unknown",
  "Балыкесир": "Unknown",
  "Piura": "Unknown",
  "Jabrayil": "Unknown",
  "кантон Аргау": "Unknown",
  "Soroksky": "Unknown",
  "Ayeyarwady": "Unknown",
  "La Paz Department": "Unknown",
  "Tachira": "Unknown",
  "Yaracuy": "Unknown",
  "Distrito Federal": "Unknown",
  "Wallonia": "Unknown",
  "Столичная территория Исламабад": "Unknown",
  "Билясуварский": "Unknown",
  "Утенский уезд": "Unknown",
  "Тукумский край": "Unknown",
  "Zagreb": "Unknown",
  "Австралийская столичная территория": "Unknown",
  "Миссисипи": "Unknown",
  "Пафос": "Unknown",
  "Тегеран": "Unknown",
  "Никосия": "Unknown",
  "Botoshani": "Unknown",
  "Астаринский": "Unknown",
  "Брюссельский столичный регион": "Unknown",
  "Саласпилсский край": "Unknown",
  "Мухафаза Хомс": "Unknown",
  "Мекленбург — Передняя Померания": "Unknown",
  "Иерусалимский округ": "Unknown",
  "Atyrauskaya": "Unknown"
}


In [27]:
d24={
  "Бурса": "Unknown",
  "Ayacucho": "Unknown",
  "Таурагский уезд": "Unknown",
  "Lesser Poland": "Unknown",
  "Систан и Белуджистан": "Unknown",
  "Alba": "Unknown",
  "Vaslui County": "Unknown",
  "Balakensky": "Unknown",
  "Porto": "Unknown",
  "Шяуляйский уезд": "Unknown",
  "Западно-Поморское воеводство": "Unknown",
  "Алуксненский край": "Unknown",
  "Zacatecas City": "Unknown",
  "Sigulda Municipality": "Unknown",
  "Kraslava Municipality": "Unknown",
  "Исфахан": "Unknown",
  "Durango": "Unknown",
  "Ольгин": "Unknown",
  "Ogrsky kray": "Unknown",
  "Любушское воеводство": "Unknown",
  "Кинтана-Роо": "Unknown",
  "Los-Lagos": "Unknown",
  "Istanbul": "Unknown",
  "Джалилабадский": "Unknown",
  "Дамаск": "Unknown",
  "Дубоссарский": "Unknown",
  "Пуэрто-Плата": "Unknown",
  "Santa Fe": "Unknown",
  "Sicily": "Unknown",
  "Балакенский": "Unknown",
  "San Luis Potosí City": "Unknown",
  "Nuevo Leon": "Unknown",
  "O’Khiggins": "Unknown",
  "Кхонкэн": "Unknown",
  "Silistra": "Unknown",
  "Gabrovskaya": "Unknown",
  "Medea": "Unknown",
  "Megimurje": "Unknown",
  "Cantabria": "Unknown",
  "Administrativno-territorialnye edinitsy levoberezhya Dnestra": "Unknown",
  "Gagauziya": "Unknown"
}


In [28]:
d25={
  "Арад": "Unknown",
  "Chimishliysky": "Unknown",
  "Almatinskaya": "Unknown",
  "Zhetysuskaya": "Unknown",
  "Северный округ": "Northen Russians",
  "Мариямпольский уезд": "Unknown",
  "Мерсин": "Unknown",
  "Kluzh": "Unknown",
  "Сантьяго": "Unknown",
  "Бардинский": "Unknown",
  "Severo-Kazakhstanskaya": "Unknown",
  "Измир": "Unknown",
  "Брэила": "Unknown",
  "Цюрих": "Unknown",
  "Tbilisi": "Unknown",
  "Исмаиллинский": "Unknown",
  "Гватемала": "Unknown",
  "О-де-Франс": "Unknown",
  "Puerto Limón": "Unknown",
  "Cartago": "Unknown",
  "Dobrich": "Unknown",
  "Zapadny bereg reki Iordan": "Unknown",
  "Zarqa": "Unknown",
  "Dzhurdzhu": "Unknown",
  "Nyuble": "Unknown",
  "San Salvador": "Unknown",
  "Amman": "Unknown",
  "Tuscany": "Unknown",
  "Liguria": "Unknown",
  "Tokat Province": "Unknown",
  "Веллингтон": "Unknown",
  "Берн": "Unknown",
  "Оран": "Unknown",
  "Закатальский": "Unknown",
  "South Savo": "Unknown",
  "Diyarbakır": "Unknown",
  "Bitlis": "Unknown",
  "Кюрдамирский": "Unknown",
  "Michigan": "Unknown",
  "Veracruz": "Unknown",
  "Arkansas": "Unknown",
  "Illinois": "Unknown",
  "Indiana": "Unknown",
  "Зальцбург": "Unknown",
  "Дорнод": "Unknown",
  "Anzoategui": "Unknown",
  "Шотландия": "Unknown",
  "Volynskaya": "Ukranians",
  "Vorumaa": "Unknown",
  "Kagulsky": "Unknown",
  "Sinop": "Unknown"
}


In [29]:
d26={
  "Кордова": "Unknown",
  "Donetskaya Narodnaya": "Ukranians",
  "Silesia": "Unknown",
  "Гамбург": "Unknown",
  "Хараре": "Unknown",
  "North Karelia": "Unknown",
  "Кельбаджарский": "Unknown",
  "Novgorodskaya": "Northen Russians",
  "Michoakan": "Unknown",
  "Chihuahua City": "Unknown",
  "Loja": "Unknown",
  "La Paz": "Unknown",
  "Ярдымлинский": "Unknown",
  "Bremen": "Unknown",
  "Tacna": "Unknown",
  "Federatsiya Bosnii i Gertsegoviny": "Unknown",
  "Styria": "Unknown",
  "Южная Каролина": "Unknown",
  "Токио": "Unknown",
  "Дьёр-Мошон-Шопрон": "Unknown",
  "Podlasie": "Unknown",
  "Sulaymaniyah": "Unknown",
  "Tarapacá": "Unknown",
  "Utensky uezd": "Unknown",
  "Сальянский": "Unknown",
  "Amasya": "Unknown",
  "Kastamonu": "Unknown",
  "Центральный Лусон": "Unknown",
  "Ordu": "Unknown",
  "Trencin Region": "Unknown",
  "Zhilinsky kray": "Unknown",
  "Синьцзян": "Unknown",
  "Samtskhe-Javakheti": "Unknown",
  "Бужумбура-Мэри": "Unknown",
  "Basque Country": "Unknown",
  "Beqaa": "Unknown",
  "Рио-де-Жанейро": "Unknown",
  "Ахсуйский": "Unknown",
  "Armavirskaya": "Southern Russians",
  "Van": "Unknown",
  "Тертерский": "Unknown",
  "Texas": "Unknown",
  "Louisiana": "Unknown",
  "Connacht": "Unknown",
  "Овернь — Рона — Альпы": "Unknown",
  "Кедабекский": "Unknown",
  "Castille-La Mancha": "Unknown",
  "Prizren": "Unknown",
  "Трнавский край": "Unknown",
  "Karlovac": "Unknown",
  "Bangkok": "Unknown",
  "Анкара": "Unknown",
  "Sakha /Yakutiya/": "Northen Russians",
  "Каир": "Unknown",
  "Южно-Курземский край": "Unknown"
}


In [30]:
# Assuming you have d1, d2, ..., d26
all_dictionaries = {**d1, **d2, **d3, **d4, **d5, **d6, **d7, **d8, **d9, **d10,
                    **d11, **d12, **d13, **d14, **d15, **d16, **d17, **d18, **d19,
                    **d20, **d21, **d22, **d23, **d24, **d25, **d26}


In [31]:
# Replace NaN with "Unknown" and map values using the dictionary
metadata['node_birthplace_region'] = metadata['node_birthplace_region'].fillna("NaN").map(all_dictionaries).fillna("Unknown")
metadata['ancestor_birthplace_region'] = metadata['ancestor_birthplace_region'].fillna("NaN").map(all_dictionaries).fillna("Unknown")


In [32]:
metadata

,node_id,ancestor_id,ancestor_level,through,node_birthplace_region,ancestor_gender,ancestor_birthplace_region,node_ibd_graph_id,ancestor_ibd_graph_id
0,2668455,2988210,0,0.0,Unknown,Female,Unknown,135785,-1
1,2668455,3533540,1,1.0,Unknown,Male,Unknown,135785,-1
2,2668455,4428401,1,0.0,Unknown,Female,Unknown,135785,-1
3,2668455,1652696,0,1.0,Unknown,Male,Unknown,135785,-1
4,2668455,2648119,1,11.0,Unknown,Male,Unknown,135785,-1
...,...,...,...,...,...,...,...,...,...
22907453,5234476,1968842,1,11.0,Northen Russians,Male,Unknown,-1,-1
22907454,5234476,1174620,1,10.0,Northen Russians,Female,Unknown,-1,-1
22907455,4008901,2395565,0,0.0,Unknown,Female,Northen Russians,-1,-1
22907456,4008901,4966205,1,1.0,Unknown,Male,Unknown,-1,-1


my implementation

In [33]:
unique_node_ibd_ids = metadata['node_ibd_graph_id'].unique()
unique_node_ibd_ids = unique_node_ibd_ids[unique_node_ibd_ids != -1]

In [34]:
np.isnan(unique_node_ibd_ids).any()

False

In [40]:
metadata[metadata['node_id'] == 2668455]

,node_id,ancestor_id,ancestor_level,through,node_birthplace_region,ancestor_gender,ancestor_birthplace_region,node_ibd_graph_id,ancestor_ibd_graph_id
0,2668455,2988210,0,0.0,Unknown,Female,Unknown,135785,-1
1,2668455,3533540,1,1.0,Unknown,Male,Unknown,135785,-1
2,2668455,4428401,1,0.0,Unknown,Female,Unknown,135785,-1
3,2668455,1652696,0,1.0,Unknown,Male,Unknown,135785,-1
4,2668455,2648119,1,11.0,Unknown,Male,Unknown,135785,-1
5,2668455,2235711,1,10.0,Unknown,Female,Unknown,135785,-1


In [57]:
node_region = dict()

with tqdm(total=len(unique_node_ibd_ids), desc="Processing") as pbar:
    for graph_node in unique_node_ibd_ids:
        subdf = metadata[metadata['node_ibd_graph_id'] == graph_node]

        node_region[graph_node] = []

        anchor_nodes = subdf['node_id'].unique()
        leaves_nodes = subdf['ancestor_id'].unique()

        while True:
            for node in anchor_nodes:
                subdf = metadata[metadata['node_id'] == node]
                region = subdf['node_birthplace_region'].unique()
                assert len(region) == 1
                node_region[graph_node].append(region[0])

            for node in leaves_nodes:
                subdf = metadata[metadata['ancestor_id'] == node]
                region = subdf['ancestor_birthplace_region'].unique()
                
                assert len(region) == 1
                node_region[graph_node].append(region[0])

            anchor_nodes = leaves_nodes.copy()
            remove_mask = []
            leaves_nodes = []
            for node in anchor_nodes:
                subdf = metadata[metadata['node_id'] == node]
                if subdf.shape[0] == 0:
                    remove_mask.append(False)
                else:
                    remove_mask.append(True)
                    leaves_nodes += subdf['ancestor_id'].unique().tolist()

            anchor_nodes = anchor_nodes[remove_mask]
            leaves_nodes = np.array(leaves_nodes)

            if len(leaves_nodes) == 0:
                break

        node_region[graph_node] = np.unique(node_region[graph_node])
        if len(node_region[graph_node]) > 1:
            del node_region[graph_node]
        elif 'Unknown' in node_region[graph_node]:
            del node_region[graph_node]

        pbar.update(1)
        pbar.set_postfix(
                found_nodes=f"{len(node_region)}",
            )

Processing:  19%|█▉        | 2407/12414 [2:52:01<11:55:11,  4.29s/it, found_nodes=0]   


KeyboardInterrupt: 

In [58]:
import pandas as pd

# Example of the function implementation
def process_metadata(metadata):
    # Dictionary to store results
    result = {}

    # Step 1: Identify unique elements in `node_ibd_graph_id`, excluding -1
    unique_node_ibd_ids = metadata['node_ibd_graph_id'].unique()
    unique_node_ibd_ids = unique_node_ibd_ids[unique_node_ibd_ids != -1]  # Exclude -1

    # Step 2: Process each unique element
    for node_ibd_id in unique_node_ibd_ids:
        # Select rows with the specific `node_ibd_graph_id`
        sub_df = metadata[metadata['node_ibd_graph_id'] == node_ibd_id]

        # Get unique values from `node_birthplace_region` column
        unique_regions = sub_df['node_birthplace_region'].dropna().unique()

        # If there is exactly one unique value, add it to the result
        if len(unique_regions) == 1 and unique_regions[0] != 'Unknown':
            result[node_ibd_id] = unique_regions[0]

    return result

# Example usage with a hypothetical metadata DataFrame
# metadata = pd.read_csv('your_file.csv')  # Load your data
result_dict = process_metadata(metadata)

# Print or use the result dictionary
print(result_dict)


{59004: 'Northen Russians', 71785: 'Northen Russians', 126555: 'Ukranians', 16630: 'Northen Russians', 137038: 'Northen Russians', 50935: 'Northen Russians', 92741: 'Northen Russians', 129386: 'Northen Russians', 93849: 'Northen Russians', 140801: 'Northen Russians', 156025: 'Northen Russians', 145425: 'Northen Russians', 92201: 'Northen Russians', 125442: 'Northen Russians', 6996: 'Northen Russians', 102312: 'Northen Russians', 32370: 'Northen Russians', 5611: 'Northen Russians', 148933: 'Northen Russians', 11619: 'Northen Russians', 9242: 'Northen Russians', 98490: 'Northen Russians', 139777: 'Southern Russians', 90575: 'Northen Russians', 82853: 'Northen Russians', 145557: 'Northen Russians', 53143: 'Northen Russians', 70498: 'Northen Russians', 38965: 'Northen Russians', 144971: 'Northen Russians', 47875: 'Northen Russians', 48044: 'Northen Russians', 119691: 'Northen Russians', 523: 'Northen Russians', 116064: 'Northen Russians', 90614: 'Northen Russians', 97005: 'Northen Russians

In [61]:
np.unique(list(result_dict.values()), return_counts=True)

(array(['Belarusians', 'Northen Russians', 'Southern Russians',
        'Ukranians'], dtype='<U17'),
 array([  4, 270,  23,  16]))

GPT implementation

In [36]:
import pandas as pd
from collections import defaultdict

def find_unique_region_ids(metadata):
    """
    For each unique node_ibd_graph_id (excluding -1), this function:
    - Builds a global tree representation from the entire `metadata`.
    - Finds all nodes/ancestors associated with that node_ibd_graph_id's initial nodes.
    - Collects all birthplace regions from the entire connected tree of nodes and ancestors.
    - If exactly one unique non-"Unknown" region is found, stores node_ibd_graph_id -> region in the result.
    - If multiple regions or "Unknown" is encountered, skips that node_ibd_graph_id.
    
    Returns:
        dict: {node_ibd_graph_id: region}
    """

    # Filter out -1 from node_ibd_graph_id
    unique_node_ibd_ids = metadata['node_ibd_graph_id'].unique()
    unique_node_ibd_ids = unique_node_ibd_ids[unique_node_ibd_ids != -1]

    # ------------------------------------------------------------
    # Build global mappings from the entire dataset
    # ------------------------------------------------------------
    # node_to_ancestors: node_id -> list of ancestor_ids
    node_to_ancestors = defaultdict(list)

    # We'll store regions in sets to handle multiple entries safely
    node_region_map = defaultdict(set)
    ancestor_region_map = defaultdict(set)

    for _, row in metadata.iterrows():
        node_id = row['node_id']
        ancestor_id = row['ancestor_id']
        node_region = row['node_birthplace_region']
        ancestor_region = row['ancestor_birthplace_region']

        # Build node -> ancestor relationship
        if pd.notna(ancestor_id):
            node_to_ancestors[node_id].append(ancestor_id)

        # Store regions for node_id
        if pd.notna(node_id) and pd.notna(node_region):
            node_region_map[node_id].add(node_region)
        
        # Store regions for ancestor_id
        if pd.notna(ancestor_id) and pd.notna(ancestor_region):
            ancestor_region_map[ancestor_id].add(ancestor_region)

    # ------------------------------------------------------------
    # Helper function to add regions from a set of IDs to all_regions
    # ------------------------------------------------------------
    def add_regions_from_map(id_set, region_map, all_regions):
        for i in id_set:
            if i in region_map:
                all_regions.update(region_map[i])

    # ------------------------------------------------------------
    # For each node_ibd_id, determine its entire tree of nodes/ancestors
    # ------------------------------------------------------------
    result = {}

    # We'll group by node_ibd_graph_id just to get the initial nodes of that graph_id
    grouped = metadata.groupby('node_ibd_graph_id')

    for node_ibd_id in unique_node_ibd_ids:
        # subdf is used only to find initial node_ids for this node_ibd_id
        subdf = grouped.get_group(node_ibd_id)
        
        # Start traversal from all node_ids found in this node_ibd_id
        current_level_node_ids = set(subdf['node_id'].unique())

        # We'll keep track of visited nodes and ancestors to avoid loops
        visited_node_ids = set()
        visited_ancestor_ids = set()

        # all_regions will store all distinct regions encountered in the entire tree
        all_regions = set()

        # Collect regions from initial nodes
        add_regions_from_map(current_level_node_ids, node_region_map, all_regions)

        # BFS-like traversal:
        # We'll go upward (node_id -> ancestor_id) and if ancestor_ids appear as node_ids,
        # continue from them too.
        while current_level_node_ids:
            visited_node_ids.update(current_level_node_ids)

            # Find all ancestors for current node_ids
            ancestor_ids = set()
            for nid in current_level_node_ids:
                if nid in node_to_ancestors:
                    ancestor_ids.update(node_to_ancestors[nid])

            # Add ancestor regions
            add_regions_from_map(ancestor_ids, ancestor_region_map, all_regions)
            visited_ancestor_ids.update(ancestor_ids)

            # Ancestors can act as nodes in next iteration if they appear as node_ids themselves
            # That is, if they appear in node_region_map or have their own ancestors
            possible_next_nodes = (
                ancestor_ids.intersection(node_region_map.keys()) |
                ancestor_ids.intersection(node_to_ancestors.keys())
            )

            # Filter out those we've visited already to prevent cycles
            next_level_node_ids = {x for x in possible_next_nodes if x not in visited_node_ids}

            # Add regions for these new node_ids
            add_regions_from_map(next_level_node_ids, node_region_map, all_regions)

            # Move to the next level of BFS
            current_level_node_ids = next_level_node_ids

        # Remove NaNs if any (though we shouldn't have them if we filled them before)
        all_regions = {r for r in all_regions if pd.notna(r)}

        # Check if "Unknown" is present. If yes, skip this node_ibd_id
        if "Unknown" in all_regions:
            continue

        # We must have exactly one unique region
        if len(all_regions) == 1:
            unique_region = next(iter(all_regions))
            # Store in result
            result[node_ibd_id] = unique_region

    return result

# Example usage (assuming `metadata` is your DataFrame):
final_result = find_unique_region_ids(metadata)
# print(final_result)


In [38]:
final_result

{13001: 'Northen Russians', 66700: 'Northen Russians'}